Open Source Software Project
========================

Traveling Santa 2018 - Prime Paths
-----------------------------------------

based on Andre Sionek, Object Oriented Santa’s Route + Concorde Solver, https://www.kaggle.com/andresionek/object-oriented-santa-s-route-concorde-solver

In [1]:
# import libs
import io
import base64
import time
import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from concorde.tsp import TSPSolver

import pylab as pl
from matplotlib import collecgtions as mc
from IPython.display import display, HTML

ModuleNotFoundError: No module named 'concorde.tsp'

City class
------------

입력 데이터로부터 받을 수 있는 데이터
x: x축 좌표
y: y축 좌표
id: 도시 번호

두 좌표 정보를 조합해 coord: (x, y)의 터플형 데이터를 만들 수 있다.

어떤 도시에 한 번 방문 했음을 표시하기 위해 boolean 타입 is_visited, 초기값 False

어떤 도시가 프라임도시인지 체크하기 위해 boolean 타입 is_prime 선언 후 check_prime()을 통해 초기화

__init__(): 객체 초기화
check_prime(): 도시 ID가 소수인지 판단하여 프라임도시 여부 확인
visit(): 도시를 방문하고 방문 여부 True로 변경
__repr__(): 객체의 정보 출력
    CityId: 도시 번호
    Coordinates: 좌표정보
    Is prime: 프라임(소수) 도시 여부
    Is visited: 방문 여부

In [ ]:
# City class
class City:   
    def __init__(self, id_, x, y):
        self.x = x
        self.y = y
        self.coord = (x, y)
        self.id = id_
        self.is_visited = False
        self.is_prime = self.check_prime()
    
    # 프라임 도시인지 체크
    def check_prime(self):
        if self.id % 2 == 0 and self.id > 2: 
            return False
        return all(self.id % i for i in range(3, int(math.sqrt(self.id)) + 1, 2))
    
    # 방문한 곳 체크
    def visit(self):
        self.is_visited = True
        
    # City 객체의 정보를 출력하는 용도
    def __repr__(self):
        fmt_str = 'CityId: {} \nCoordinates: {}\nIs prime: {}\nIs visited: {}'
        return fmt_str.format(self.id, self.coord, self.is_prime, self.is_visited)

World class
-
Dictionary 컬렉션

In [ ]:
class World:
    def __init__(self):
        self.cities = dict()
        self.df = pd.DataFrame()
    
    # City 객체 하나를 World에 추가
    def add(self, city):
        if isinstance(city, City):
            # checks if the object type added to the world is correct
            self.cities[city.id] = city
        else:
            raise TypeError ('city must be a "__main__.City" object!')
        self.update_df()
    
    # City 객체 하나를 World에서 제거
    def remove_by_id(self, id_):
        self.cities.pop(id_)
    
    # 데이터프레임에서 도시 리스트 불러오기
    def bulk_add_city(self, dataframe):
        dataframe['city'] = dataframe.apply(lambda x: City(x.iloc[0], x.iloc[1], x.iloc[2]), axis=1)
        dataframe = dataframe.drop(['X','Y'], axis=1)
        dataframe = dataframe.set_index('CityId')
        cities = dataframe.to_dict()['city']
        self.cities.update(cities)
        self.update_df()
    
    # Gets the quantity of cities in the world
    def size(self):
        return self.df.shape[0]
    
    # World에서 프라임 도시 얻기
    def primes(self):
        return [c for idx, c in self.cities.items() if c.is_prime]
    
    # World에서 프라임 도시가 아닌 도시 얻기
    def nonprimes(self):
        return [c for idx, c in self.cities.items() if not c.is_prime]
    
    # 북극(시작점, CityId == 0) 얻기
    def north_pole(self):
        return self.cities[0]
    
    # World에 있는 모든 City의 Id 값(key) 얻기
    def ids(self):
        return list(self.cities.keys())
    
    # Id(key) 값으로 City 얻기
    def get_city(self, id_):
        return self.cities[id_]
    
    # 도시 좌표를 dataframe으로 가져오기
    def update_df(self):
        dataframe = pd.DataFrame.from_dict(world.cities, orient='index', columns=['city'])
        dataframe['x'] = dataframe.apply(lambda x: x.iloc[0].x, axis=1)
        dataframe['y'] = dataframe.apply(lambda x: x.iloc[0].y, axis=1)
        dataframe['id'] = dataframe.apply(lambda x: x.iloc[0].id, axis=1)
        dataframe['is_prime'] = dataframe.apply(lambda x: x.iloc[0].is_prime, axis=1)
        self.df = dataframe
    
    # World 그래프 그리기. 북극을 빨간색으로 표시하고, 옵션으로 프라임 도시를 파란 색으로 표시
    def plot(self, show_primes=False):
        self.df.plot.scatter(x='x', y='y', s=0.07, figsize=(15, 10), c='green', alpha=0.6)

        if show_primes:
            coords = self.df[self.df['is_prime']]
            plt.scatter(x=coords.x, y=coords.y, c='blue', s=0.1, alpha=0.6)
        
        plt.scatter(self.north_pole().x, self.north_pole().y, c='red', s=16)
        plt.axis('off')
        
        return plt.show()

데이터프레임 불러오기

In [ ]:
df = pd.read_csv('./data/cities.csv')
df.head()

In [ ]:
df.shape # the dataset shape

In [ ]:
world = World() # creates a new empty world
world.cities

In [ ]:
world.bulk_add_city(df) # adds all cities from the dataframe into the world

In [ ]:
world.size() # 도시의 총 갯수

In [ ]:
world.north_pole().coord # 북극(CityId == 0)의 좌표 확인

In [ ]:
len(world.primes()) # 프라임 도시의 갯수

In [ ]:
world.plot() # 그래프

In [ ]:
world.plot(show_primes=True) # 프라임도시를 별도 색으로 표기한 그래프

경로 찾기
---------

In [ ]:
class Route():
    def __init__(self, world):
        # Init path with just the North Pole
        self.world = world
        self.stops = [self.world.north_pole()]
        self.tenth_steps = []
        self.prime_tenth_steps = self.get_prime_tenth_steps()
    
    # 두 도시간 2D 유클리드 거리 계산
    @staticmethod
    def distance(a, b):
        return math.sqrt((a.x - b.x) ** 2 + (a.y - b.y) ** 2)
    
    # Calculates the number of stops in the path
    def size(self):
        return len(self.stops)
    
    # Add stop to path
    def add(self, ids):
        if isinstance(ids, type(list())):
            for id_ in ids:
                self.stops.append(self.world.get_city(id_))
        elif isinstance(ids, type(int())):
            self.stops.append(self.world.get_city(ids))
    
    # Adds all cities in the world to the route
    def add_world(self):
        self.add(list(range(1, self.world.size())))
    
    # 패널티를 포함한 전체 경로의 비용 계산
    def cost(self):
        dist = 0
        l = self.size()
        for idx, stop in enumerate(self.stops):
            curr_ = stop
            step_number = idx + 1
            if step_number < l:
                next_ = self.stops[step_number]
            if step_number % 10 == 0 and not curr_.is_prime:
                dist += self.distance(curr_, next_) * 1.1
            else:
                dist += self.distance(curr_, next_)
        return dist
    
    # 패널티를 제외한 경로의 비용 계산
    def euclidian_cost(self):
        dist = 0
        l = self.size()
        for idx, stop in enumerate(self.stops):
            curr_ = stop
            step_number = idx + 1
            if step_number < l:
                next_ = self.stops[step_number]
            dist += self.distance(curr_, next_)
        return dist
    
    # 프라임도시에 가지 않아서 생긴 패널티 총량
    def penalty(self):
        return self.cost() - self.euclidian_cost()
    
    # 10번째마다 방문한 도시들 구하기
    def get_tenth_steps(self):
        self.tenth_steps = [c for s,c in enumerate(self.stops) if (s + 1) % 10 == 0]
        return self.tenth_steps
    
    # 10번째마다 방문한 도시 중 프라임 도시 구하기
    def get_prime_tenth_steps(self):
        primes = self.world.primes()
        self.prime_tenth_steps = [c for c in self.tenth_steps if c in primes]
        return self.prime_tenth_steps
    
    # 방문 경로대로 순차 정렬된 도시 리스트 얻기
    def get_ids(self):
        return [c.id for c in self.stops]

    # 콩코드로 문제 풀이, 패널티 제외
    def concorde(self, time):
        x = [c.x for c in self.stops[:-1]]
        y = [c.y for c in self.stops[:-1]]

        # Instantiate solver
        solver = TSPSolver.from_data(x, y, norm="EUC_2D")
        
        # solve
        tour_data = solver.solve(time_bound=float(time), verbose=True, random_seed=42)
        
        # Reorder the route with concorde solution
        order = np.append(tour_data.tour,[0])
        new_route = [self.stops[i] for i in order]
        self.stops = new_route
    
    # 경로를 그래프로 그리기
    def plot(self, show_primes=False, show_10th_step=False, show_intersection=False):
        lwidth, lalpha = 0.4, 0.6
        lines = [[self.stops[i].coord,self.stops[i + 1].coord] for i in range(0, self.size() - 1)]
        lc = mc.LineCollection(lines, linewidths=lwidth, alpha=lalpha, colors='red')
        fig, ax = pl.subplots(figsize=(20,20))
        
        ax.set_aspect('equal')
        ax.add_collection(lc)
        ax.autoscale()   
        
        if show_primes:
            # get coords of prime cities
            coords = self.world.df[self.world.df['is_prime']]
            # plot
            pc = plt.scatter(x=coords.x, y=coords.y, c='green', s=3, alpha=0.3) 
            ax.add_collection(pc)
        
        if show_10th_step:
            # get coords of each 10th stop
            coords = [c.coord for c in self.tenth_steps]
            coords = pd.DataFrame(coords, columns=['x','y'])
            # plot 
            pc = plt.scatter(x=coords.x, y=coords.y, c='blue', s=3, alpha=0.3) 
            ax.add_collection(pc)
        
        if show_intersection:
            # get coords of each 10th stop
            coords = [c.coord for c in self.prime_tenth_steps]
            coords = pd.DataFrame(coords, columns=['x','y'])
            # plot 
            pc = plt.scatter(x=coords.x, y=coords.y, c='black', s=4, alpha=0.5) 
            ax.add_collection(pc)
            return
                
    def load(self, path):
        """
        Loads external solution from csv file
        """
        df = pd.read_csv(path)
        ids = df['Path'].values.tolist()
        self.stops = []
        for id_ in ids:
            self.stops.append(self.world.get_city(id_))
    
    # 경로 정보를 csv파일로 내보내기
    def submit(self, filename):
        dataframe = pd.DataFrame(self.stops, columns=['city'])
        dataframe['Path'] = dataframe.apply(lambda x: x.iloc[0].id, axis=1).astype(int)
        dataframe[['Path']].to_csv(filename, index=False)

In [ ]:
route = Route(world)
route.stops

In [ ]:
route.stops[0].id, route.stops[0].coord

In [ ]:
route.add_world # 경로에 모든 도시 추가(world에 추가되어 있다)
route.add(0) # 마지막 목적지 북극을 경로에 추가

In [ ]:
route.concorde(60) # 콩코드로 경로 계산

In [ ]:
route.cost() # 패널티 포함 전체 경로 비용

In [ ]:
route.euclidian_cost() # 패널티 제외 전체 경로 비용

In [ ]:
route.penalty() # 패널티 총량

In [ ]:
route.get_tenth_steps() # 10번째마다 방문한 도시 목록
route.get_prime_tenth_steps() # 그 중 프라임도시인 목록
len(route.prime_tenth_steps) / len(route.tenth_steps) # 10번째마다 방문한 도시가 프라임도시인 확률

In [ ]:
route.plot()